<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task-6-(Work-by:-Nakul-and-Yutong)" data-toc-modified-id="Task-6-(Work-by:-Nakul-and-Yutong)-1">Task 6 (Work by: Nakul and Yutong)</a></span><ul class="toc-item"><li><span><a href="#1.-Data-processing-(Work-by:-Nakul)" data-toc-modified-id="1.-Data-processing-(Work-by:-Nakul)-1.1">1. Data processing (Work by: Nakul)</a></span><ul class="toc-item"><li><span><a href="#Human-beta-chain" data-toc-modified-id="Human-beta-chain-1.1.1">Human beta chain</a></span></li><li><span><a href="#Drop-duplicates" data-toc-modified-id="Drop-duplicates-1.1.2">Drop duplicates</a></span></li><li><span><a href="#CDR3-length-$\in$-[10,20]" data-toc-modified-id="CDR3-length-$\in$-[10,20]-1.1.3">CDR3 length $\in$ [10,20]</a></span></li><li><span><a href="#Epitopes-binds-to-10-more-TCRs" data-toc-modified-id="Epitopes-binds-to-10-more-TCRs-1.1.4">Epitopes binds to 10 more TCRs</a></span></li></ul></li><li><span><a href="#2.-TEINet-(Work-by:-Yutong)" data-toc-modified-id="2.-TEINet-(Work-by:-Yutong)-1.2">2. TEINet (Work by: Yutong)</a></span><ul class="toc-item"><li><span><a href="#create-csv-files-for-train-and-test-data" data-toc-modified-id="create-csv-files-for-train-and-test-data-1.2.1">create csv files for train and test data</a></span></li><li><span><a href="#2.1-The-required-software-dependencies:" data-toc-modified-id="2.1-The-required-software-dependencies:-1.2.2">2.1 The required software dependencies:</a></span></li><li><span><a href="#2.2-USE-ANACONDA-PROMPT:" data-toc-modified-id="2.2-USE-ANACONDA-PROMPT:-1.2.3">2.2 USE ANACONDA PROMPT:</a></span><ul class="toc-item"><li><span><a href="#2.2.1-Metric:-Brier-Score" data-toc-modified-id="2.2.1-Metric:-Brier-Score-1.2.3.1">2.2.1 Metric: Brier Score</a></span></li><li><span><a href="#2.2.2-Train-our-own-model:" data-toc-modified-id="2.2.2-Train-our-own-model:-1.2.3.2">2.2.2 Train our own model:</a></span></li></ul></li><li><span><a href="#2.3-Use-trained-TEINet-models:-teinet_data.pth" data-toc-modified-id="2.3-Use-trained-TEINet-models:-teinet_data.pth-1.2.4">2.3 Use trained TEINet models: teinet_data.pth</a></span></li></ul></li></ul></li></ul></div>

# Task 6 (Work by: Nakul and Yutong)

## 1. Data processing (Work by: Nakul)

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('vdjdb.csv')

In [3]:
dataset = dataset.iloc[:, list(range(0, 12)) + [16]]

In [4]:
sample_dataset = dataset[['cdr3','antigen.epitope','vdjdb.score','species','antigen.species','gene','mhc.class']]

In [5]:
sample_dataset = dataset[['cdr3','antigen.epitope','vdjdb.score','species','antigen.species','gene','mhc.class']]

### Human beta chain

In [6]:
sample_dataset = sample_dataset[(sample_dataset['gene'] == 'TRB') & (sample_dataset['species'] == 'HomoSapiens')]

### Drop duplicates

In [7]:
duplicate_indexes = sample_dataset[sample_dataset.duplicated(keep='last')].index
sample_dataset.drop(index=duplicate_indexes,inplace=True)

### CDR3 length $\in$ [10,20]

In [8]:
sample_dataset['CDR3_Length'] = sample_dataset.apply(lambda row: len(row['cdr3']),axis=1)

In [9]:
sample_dataset = sample_dataset[(sample_dataset['CDR3_Length'] >= 10) & (sample_dataset['CDR3_Length'] <= 20)]

### Epitopes binds to 10 more TCRs

In [10]:
group_antigen_cdr3 = sample_dataset.groupby(['antigen.epitope'])['cdr3'].unique().to_frame('cdr3_found').reset_index()

In [11]:
group_antigen_cdr3['cdr3_found_count'] = group_antigen_cdr3['cdr3_found'].apply(lambda x:len(x))

In [12]:
group_antigen_cdr3_count = group_antigen_cdr3.groupby(by=['cdr3_found_count']).size().to_frame('count').reset_index()

In [13]:
group_antigen_cdr3_count.sort_values('count',ascending=False)

,cdr3_found_count,count
0,1,450
1,2,182
2,3,111
3,4,69
4,5,47
...,...,...
69,101,1
70,103,1
71,110,1
72,112,1


In [14]:
antigen_needed_list = group_antigen_cdr3[group_antigen_cdr3['cdr3_found_count'] >= 10]['antigen.epitope']

In [15]:
sample_dataset = sample_dataset[sample_dataset['antigen.epitope'].isin(antigen_needed_list)]

In [16]:
# sample_dataset.to_csv('pre_processed_vdj_db.csv',index=False)

In [17]:
model_df = sample_dataset[['cdr3','antigen.species','mhc.class','antigen.epitope','vdjdb.score']]
model_df

,cdr3,antigen.species,mhc.class,antigen.epitope,vdjdb.score
1,CASSYLPGQGDHYSNQPQHF,HIV-1,MHCI,FLKEKGGL,2
2,CASSFEAGQGFFSNQPQHF,HIV-1,MHCI,FLKEKGGL,2
4,CASSFEPGQGFYSNQPQHF,HIV-1,MHCI,FLKEKGGL,2
6,CASSYEPGQVSHYSNQPQHF,HIV-1,MHCI,FLKEKGGL,2
8,CASSALASLNEQFF,HIV-1,MHCI,FLKEKGGL,2
...,...,...,...,...,...
92748,CASSIRSTDTQYF,Wheat,MHCII,PQPELPYPQPQL,0
92758,CASSLRSTDTQYF,Wheat,MHCII,PQPELPYPQPQL,0
92760,CASSFRSTDTQYF,Wheat,MHCII,PQPELPYPQPQL,0
92762,CASSVRSTDTQYF,Wheat,MHCII,PQPELPYPQPQL,0


## 2. TEINet (Work by: Yutong)

Please download the TEINet (https://github.com/jiangdada1221/TEINet) in the same folder as this notebook.

We have Modified the original train.py predict.py. Please replace the files "train.py predict.py" by the files provided in the same folder as this notebook.

### create csv files for train and test data

In [20]:
import pandas as pd
import os
import pandas as pd
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)
random_seed = 42

## Taking VDJdb scores=0 as negative 
train_test_data = model_df[['cdr3', 'antigen.epitope','vdjdb.score']]
train_test_data.rename(columns={'cdr3': 'CDR3.beta', 'antigen.epitope': 'Epitope'}, inplace=True)

train_test_data['Label'] = 1
# train_test_data['Label'] = train_test_data['vdjdb.score'].apply(lambda x: 0 if x == 0 else 1)

print(train_test_data)
train_test_data.to_csv(os.path.join(data_dir,'predict_sample.csv'),index=False)

                  CDR3.beta       Epitope  vdjdb.score  Label
1      CASSYLPGQGDHYSNQPQHF      FLKEKGGL            2      1
2       CASSFEAGQGFFSNQPQHF      FLKEKGGL            2      1
4       CASSFEPGQGFYSNQPQHF      FLKEKGGL            2      1
6      CASSYEPGQVSHYSNQPQHF      FLKEKGGL            2      1
8            CASSALASLNEQFF      FLKEKGGL            2      1
...                     ...           ...          ...    ...
92748         CASSIRSTDTQYF  PQPELPYPQPQL            0      0
92758         CASSLRSTDTQYF  PQPELPYPQPQL            0      0
92760         CASSFRSTDTQYF  PQPELPYPQPQL            0      0
92762         CASSVRSTDTQYF  PQPELPYPQPQL            0      0
92764         CASSLRYTDTQYF  PQPELPYPQPQL            0      0

[39524 rows x 4 columns]


D:\TEMP\ipykernel_22924\487043245.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data.rename(columns={'cdr3': 'CDR3.beta', 'antigen.epitope': 'Epitope'}, inplace=True)
D:\TEMP\ipykernel_22924\487043245.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_test_data['Label'] = train_test_data['vdjdb.score'].apply(lambda x: 0 if x == 0 else 1)


In [21]:
# Randomly select 20% of the records as the test set
test_data = train_test_data.sample(frac=0.2, random_state = random_seed)

# Use the remaining records as a training set
train_data = train_test_data.drop(test_data.index)

train_data.to_csv(os.path.join(data_dir, 'train.csv'), index=False)
test_data.to_csv(os.path.join(data_dir, 'test.csv'), index=False)

### 2.1 The required software dependencies:

```
Numpy
matplotlib
tqdm
pandas
scikit-learn
scipy
torch >= 1.1.0 (Tested on 1.8.0)
Levenshtein
```


### 2.2 USE ANACONDA PROMPT:

**REMEMBER to change file path to current folder.**

**Taking all VDJdb scores as positive**



#### 2.2.1 Metric: Brier Score

The Brier Score is a measure used to quantify the accuracy of probabilistic predictions, particularly for binary outcomes. It calculates the mean squared difference between the predicted probabilities and the actual outcomes. A Brier Score of 0 indicates perfect accuracy, whereas a score of 1 represents the worst possible prediction. 

####  2.2.2 Train our own model:

**Test records: 20%**

Entire train set (31619 records) and test set (7905 records) with epochs 3 

**Best Brier Score = 0.6381** 

(Takes a long time, the score is on a downward trend)


---
(pytorch_env) > python train.py --train_file data/train.csv --test_file data/test.csv --epochs 3 --model_path results/model.pth

---
using the pretrained model
Begin training
Epoch = 3
sample num = 10
dropout = 0.0
Using frequency to sample = True
cat_size = 1536

100%|████████████████████| 659/659 [44:22<00:00,  4.04s/it]

100%|████████████████████| 165/165 [00:08<00:00, 20.02it/s]

Epoch:  0

Test brier_score_loss:  0.6381142171853036

100%|████████████████████| 659/659 [47:11<00:00,  4.30s/it]

100%|████████████████████| 165/165 [00:08<00:00, 20.11it/s]

Epoch:  1

Test brier_score_loss:  0.7202708440888607

100%|████████████████████| 659/659 [48:59<00:00,  4.46s/it]

100%|████████████████████| 165/165 [00:08<00:00, 19.96it/s]

Epoch:  2

Test brier_score_loss:  0.6475847326234571


---



### 2.3 Use trained TEINet models: teinet_data.pth

**39524 records,  Brier Score = 0.2510**

---
(pytorch_env) >python predict.py --dset_path data/predict_sample.csv --save_prediction_path data/prediction.csv --model_path results/teinet_data.pth

---
100%|██████████████████████████| 309/309 [00:31<00:00,  9.84it/s]

Test brier_score_loss:  0.25106405222743194